## Imports and Setup

In [2]:
import os
import sys
sys.path.insert(0, "../utils/")
import cnn_utils
import config_utils

import pandas as pd
import numpy as np
import geopandas as gpd

import torch

pd.set_option('mode.chained_assignment', None)
%load_ext autoreload
%autoreload 2

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


## Load Configuration

In [3]:
iso = "SEN"
config_file = "configs/cnn_configs/xception.yaml"

cwd = os.path.dirname(os.getcwd())
config = config_utils.load_config(os.path.join(cwd, config_file))
config["iso_codes"] = [iso]
data, data_loader, classes = cnn_utils.load_dataset(config, phases=["train", "test"])

INFO:root:Reading file d:\Faks\5.godina\4.semestar\school_mapping\data/vectors/train\SEN_train.geojson
INFO:root:
  rurban  iso       class  count  percentage
0  urban  SEN  non_school   2544    0.784943
1  urban  SEN      school    697    0.215057
INFO:root:
                               count  percentage
rurban iso class      dataset                   
urban  SEN non_school test       480    0.740741
                      train     2064         NaN
           school     test       168    0.259259
                      train      529         NaN
INFO:root:
                    count
dataset class            
test    non_school    480
        school        168
train   non_school   2064
        school        529
INFO:root:
dataset
train    2593
test      648
Name: count, dtype: int64
INFO:root:
rurban
urban    3241
Name: count, dtype: int64
INFO:root:
iso
SEN    3241
Name: count, dtype: int64
INFO:root:
class
non_school    2544
school         697
Name: count, dtype: int64
INFO:root: Cla

## Visualize Samples

In [4]:
cnn_utils.visualize_data(data, data_loader, phase='train')

FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "c:\Users\andri\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\_utils\worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
           ^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\andri\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\_utils\fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "d:\Faks\5.godina\4.semestar\school_mapping\notebooks\../utils\cnn_utils.py", line 86, in __getitem__
    image = Image.open(filepath).convert("RGB")
            ^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\andri\AppData\Local\Programs\Python\Python312\Lib\site-packages\PIL\Image.py", line 3247, in open
    fp = builtins.open(filename, "rb")
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: 'd:\\

## Load Model

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model, criterion, optimizer, scheduler = cnn_utils.load_model(
    n_classes=len(classes),
    model_type=config["model"],
    pretrained=config["pretrained"],
    scheduler_type=config["scheduler"],
    optimizer_type=config["optimizer"],
    label_smoothing=config["label_smoothing"],
    lr=config["lr"],
    momentum=config["momentum"],
    gamma=config["gamma"],
    step_size=config["step_size"],
    patience=config["patience"],
    dropout=config["dropout"],
    device=device,
)
model

c:\Users\andri\AppData\Local\Programs\Python\Python312\Lib\site-packages\timm\models\_factory.py:117: UserWarning: Mapping deprecated model name xception to current legacy_xception.
  model = create_fn(
INFO:timm.models._builder:Loading pretrained weights from url (https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-cadene/xception-43020ad28.pth)
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-cadene/xception-43020ad28.pth" to C:\Users\andri/.cache\torch\hub\checkpoints\xception-43020ad28.pth
INFO:timm.models._builder:Missing keys (fc.weight, fc.bias) discovered while loading pretrained weights. This is expected if model is being adapted.


Xception(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act2): ReLU(inplace=True)
  (block1): Block(
    (skip): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
    (skipbn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (rep): Sequential(
      (0): SeparableConv2d(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
        (pointwise): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): SeparableConv2d(
        (conv1): Conv